# DuckDB tutorial

## Setup

First, create a Python virtual environment. The following shell command will create a virtual environment in the current directory named 'duckdb':


``
python -m venv duckdb
``
We can activate the environment using either on UNIX-like systems:

``
source duckdb/bin/activate
``

or on Microsoft Windows:

``
.\duckdb\Scripts\activate
``

This might require to [change the scripts execution policy](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.security/set-executionpolicy?view=powershell-7.5) on the host machine. In powershell, one can change it using the following cmdlet. This will allow the system to run scripts for the current process (i.e. terminal session).

``
Set-ExecutionPolicy -ExecutionPolicy AllSigned -Scope Process
``

Now we can start downloading the required libraries.

``
pip install -r requirements.txt
``

We are now ready to go.


In [15]:
import duckdb

# Will create the file if it doesn't exist
conn = duckdb.connect('duckdb.db')

query = "CREATE TABLE IF NOT EXISTS hello AS SELECT 'World'"

conn.execute(query)

fetch = "SELECT * FROM hello"

result = conn.execute(fetch).fetchall()

print(result)

[('World',)]
